In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
## denote the working directory
%cd /Users/cyliai/COMP4332

/Users/cyliai/COMP4332


In [57]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cyliai/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/cyliai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/cyliai/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [2]:
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [3]:
lemmatizer = WordNetLemmatizer()
neutral = [0,0,0]
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return neutral

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return neutral

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return neutral

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [4]:
df = pd.read_csv('train.csv')
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,dsvFUqrhytVmKXCW7bKNhA,0,2013-03-10 01:24:07,0,58nqw-MdO6EDACPaKDM59Q,5.0,"As a student, by back and neck are under const...",0,1NImHsg1kc76n_cQyTm0bg
1,6YrO-hJNof4wsx4f0YQ8yg,0,2014-12-22 17:06:23,0,eVeQtMGaB5tdCH0hKJwxKw,3.0,Stayed here for a football game at University ...,0,Oi0G3jFm2jtG2W02dZTdEQ
2,LtyoPfxpvcF_9e9wMoUi0w,0,2016-04-25 09:55:57,0,AHMHUbq0eAUOuOPFoeO5iw,4.0,"Very good salads, generous portions. I either ...",1,4jbz7cOVuV_Q7v2b3pNrLw
3,QzvLnOqwH6BIY_jCOvzuQQ,0,2017-04-06 04:18:48,0,8RDvwneSMsbpmi5UgGq60A,1.0,The experience I had with this company growing...,0,gJgPs0QXE587T9SIR6NCdQ
4,tPaweigPsXacvQT8daYT4g,0,2017-03-22 15:38:59,0,IboCoxoL0IFtJnsGPHCugg,5.0,"Easiest furniture purchase, with a great price...",0,zp8XLBJmgw55ZTlp5raK7Q


In [5]:
dfr = df.drop(['review_id','business_id','date','user_id'],axis=1)
dfr.head()

,cool,funny,stars,text,useful
0,0,0,5.0,"As a student, by back and neck are under const...",0
1,0,0,3.0,Stayed here for a football game at University ...,0
2,0,0,4.0,"Very good salads, generous portions. I either ...",1
3,0,0,1.0,The experience I had with this company growing...,0
4,0,0,5.0,"Easiest furniture purchase, with a great price...",0


In [6]:
y_train = dfr['stars']
x_train = dfr.drop(['stars'],axis=1)

In [7]:
print(x_train.head())
print(y_train.head())

   cool  funny                                               text  useful
0     0      0  As a student, by back and neck are under const...       0
1     0      0  Stayed here for a football game at University ...       0
2     0      0  Very good salads, generous portions. I either ...       1
3     0      0  The experience I had with this company growing...       0
4     0      0  Easiest furniture purchase, with a great price...       0
0    5.0
1    3.0
2    4.0
3    1.0
4    5.0
Name: stars, dtype: float64


In [8]:
def scorer(dfcol):
    ps = PorterStemmer()
    pos_score_list = []
    neg_score_list = []
    neu_score_list = []
    pos_count_list = []
    neg_count_list = []
    for each in dfcol:
        sent = each
        pos_score = 0; neg_score = 0; neu_score = 0; pos_count = 0; neg_count = 0
        token = nltk.word_tokenize(sent)
        words_data = [ps.stem(x) for x in token]
        pos_val = nltk.pos_tag(words_data)
        senti_val=[ get_sentiment(x,y) for (x,y) in pos_val]
        for eachhaha in senti_val:
            pos_score += eachhaha[0]
            neg_score += eachhaha[1]
            neu_score += eachhaha[2]
            if pos_score > neg_score:
                pos_count += 1
            elif pos_score < neg_score:
                neg_count += 1
        pos_score_list.append(pos_score)
        neg_score_list.append(neg_score)
        neu_score_list.append(neu_score)
        pos_count_list.append(pos_count)
        neg_count_list.append(neg_count)
    Texttuples = list(zip(pos_score_list,neg_score_list,neu_score_list,pos_count_list,neg_count_list))
    Textdf = pd.DataFrame(Texttuples,columns=['pos_score','neg_score','neutral_score','pos_wd_count','neg_wd_count'])
    return Textdf

In [9]:
scorer(x_train.head()['text'])

,pos_score,neg_score,neutral_score,pos_wd_count,neg_wd_count
0,0.500,1.625,8.875,9,27
1,1.125,0.000,13.875,22,0
2,0.750,0.000,5.250,23,0
3,1.375,1.000,36.625,94,0
4,0.750,0.500,5.750,24,0
